In [28]:
import pandas as pd
from datetime import date, timedelta
import urllib
import numpy as np
import altair as alt

In [29]:
# Configuration variables
today = date.today()
yesterday = date.today() - timedelta(days=1)
nhs_header_row = 15
nhs_trust_sheet_name = 'COVID19 total deaths by trust'
nhs_csv_trusts = 'nhs_csv_trusts.csv'
our_trust_codes = ['RYW','RNA','RL4','RXK','RXW','RRJ','RRK','RJE','RBK','RWP','R1A']
nhs_file_format = "https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/DIRECTORY/COVID-19-total-announced-deaths-DATE.xlsx"
nhs_file_date_format = "%-d-%B-%Y"
nhs_file_directory_format = "%Y/%m"
nhs_latest_file = False

nhs_today_file = nhs_file_format.replace("DATE",today.strftime(nhs_file_date_format))
nhs_today_file = nhs_today_file.replace("DIRECTORY",today.strftime(nhs_file_directory_format))

nhs_yesterday_file = nhs_file_format.replace("DATE",yesterday.strftime(nhs_file_date_format))
nhs_yesterday_file = nhs_yesterday_file.replace("DIRECTORY",yesterday.strftime(nhs_file_directory_format))

In [30]:
# Try the NHS URL for todays file, fallback to yesterdays
try:
  urllib.request.urlopen(nhs_today_file)
  nhs_latest_file = nhs_today_file
except urllib.error.HTTPError as e:
  try:
    nhs_latest_file = urllib.request.urlopen(nhs_yesterday_file)
    nhs_latest_file = nhs_yesterday_file
  except urllib.error.HTTPError as e:
    pass

In [31]:
nhs_latest_file

'https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2020/04/COVID-19-total-announced-deaths-9-April-2020.xlsx'

In [32]:
# Import a specific sheet from the NHS Excel file and convert to CSV
nhs_excel = pd.read_excel(nhs_latest_file, header=nhs_header_row, sheet_name=nhs_trust_sheet_name)
nhs_excel.to_csv(nhs_csv_trusts)
nhs_trusts = pd.read_csv(nhs_csv_trusts)

In [33]:
our_trusts = nhs_trusts[nhs_trusts.Code.isin(our_trust_codes)] # filter to our trusts
our_trusts = our_trusts.filter(regex='(^Code$|^Name$|[0-9]{4}-.+|^Up to.01-Mar-20$)', axis=1) # filter to only the required columns
our_trusts = our_trusts.rename(columns={"Code": "code", "Name" : "name"}) # clean up column names
our_trusts = pd.melt(our_trusts, id_vars=["code","name"], var_name="date", value_name="deaths") # un-pivot the date columns
our_trusts["date"] = our_trusts["date"].str.replace("Up to 01-Mar-20","2020-02-29 00:00:00")
our_trusts["date"] = pd.to_datetime(our_trusts.date) # clean all dates to YYYY-MM-DD

In [34]:
our_trusts.pivot_table(our_trusts, index=["code","name"], aggfunc=np.sum)

,,deaths
code,name,
R1A,WORCESTERSHIRE HEALTH AND CARE NHS TRUST,4.0
RBK,WALSALL HEALTHCARE NHS TRUST,37.0
RJE,UNIVERSITY HOSPITALS OF NORTH MIDLANDS NHS TRUST,71.0
RL4,THE ROYAL WOLVERHAMPTON NHS TRUST,126.0
RNA,THE DUDLEY GROUP NHS FOUNDATION TRUST,110.0
RRJ,THE ROYAL ORTHOPAEDIC HOSPITAL NHS FOUNDATION TRUST,1.0
RRK,UNIVERSITY HOSPITALS BIRMINGHAM NHS FOUNDATION TRUST,306.0
RWP,WORCESTERSHIRE ACUTE HOSPITALS NHS TRUST,52.0
RXK,SANDWELL AND WEST BIRMINGHAM HOSPITALS NHS TRUST,116.0


In [35]:
our_trusts["cumulative_deaths"] = our_trusts.groupby("name").cumsum()

In [36]:
our_trusts

,code,name,date,deaths,cumulative_deaths
0,RYW,BIRMINGHAM COMMUNITY HEALTHCARE NHS FOUNDATION...,2020-02-29,0.0,0.0
1,RXK,SANDWELL AND WEST BIRMINGHAM HOSPITALS NHS TRUST,2020-02-29,1.0,1.0
2,RXW,SHREWSBURY AND TELFORD HOSPITAL NHS TRUST,2020-02-29,0.0,0.0
3,RNA,THE DUDLEY GROUP NHS FOUNDATION TRUST,2020-02-29,0.0,0.0
4,RRJ,THE ROYAL ORTHOPAEDIC HOSPITAL NHS FOUNDATION ...,2020-02-29,0.0,0.0
...,...,...,...,...,...
435,RRK,UNIVERSITY HOSPITALS BIRMINGHAM NHS FOUNDATION...,2020-04-08,0.0,306.0
436,RJE,UNIVERSITY HOSPITALS OF NORTH MIDLANDS NHS TRUST,2020-04-08,2.0,71.0
437,RBK,WALSALL HEALTHCARE NHS TRUST,2020-04-08,0.0,37.0
438,RWP,WORCESTERSHIRE ACUTE HOSPITALS NHS TRUST,2020-04-08,1.0,52.0


In [37]:
alt.Chart(our_trusts).mark_line().encode(
    x='date',
    y='cumulative_deaths',
    color='name',
    tooltip='name'
)

alt.Chart(...)

In [38]:
alt.Chart(our_trusts).mark_line().encode(
    x='date',
    y='deaths',
    color='name',
    tooltip='name'
)

alt.Chart(...)

In [41]:
alt.Chart(our_trusts).mark_bar().encode(
    x='date',
    y='sum(deaths)',
    color='name',
    tooltip='name'
)

alt.Chart(...)